# MATH 405/607 

# Numerical Methods for Differential Equations

[[Instructor: Christoph Ortner]](http://www.math.ubc.ca/~ortner/)  [[CANVAS]](https://canvas.ubc.ca/courses/55324)


## Solving Large Linear Systems

This is an ultra-rapid introduction to a topic that I've neglected throughout this course, but is extremely important once you start solving "real-world problems". The idea is to give you some initial exposure and intuition on a few important ideas

* complexity of direct solvers for 2D, 3D systems
* the idea of iterative solvers (steepest descent for linear systems)
* Krylov subspace methods (Conjugate gradient method)
* Cascading multigrid (towards linear scaling!)



### Literature

There are many good lecture notes, and slides on these topics. Here are some that I used to prepare for this lecture:

* [[pdf]](https://www.cs.cmu.edu/~quake-papers/painless-conjugate-gradient.pdf) An Introduction to the Conjugate Gradient Method Without the Agonizing Pain by J. Shewchuck, 
* [[pdf]](http://www.maths.lth.se/na/courses/NUM115/NUM115-05/krylov.pdf) The Idea Behind Krylov Methods, by Ilse C.F. Ipsen and Carl D. Meyer

Cascadic Multigrid: 
* [[html]](https://link.springer.com/article/10.1007/s002110050234) The cascadic multigrid method for elliptic problems, Folkmar A. Bornemann & Peter Deuflhard , Numer. Math. 75, pages135–152(1996)
* [[pdf]](https://opus4.kobv.de/opus4-zib/files/119/SC-93-23.pdf) Deuflhard, P. Cascadic conjugate gradient methods for elliptic partial differential equations. Algorithm and numerical results.

"Proper" Multi-grid:
* [[pdf]](https://www.math.ust.hk/~mawang/teaching/math532/mgtut.pdf) A Multigrid Tutorial, by William L. Briggs
* [[pdf]](https://people.cs.kuleuven.be/~karl.meerbergen/didactiek/h03g1a/multigrid.pdf) A short multigrid tutorial, by Karl Meerbergen
* [[pdf]](http://helper.ipam.ucla.edu/publications/es2002/es2002_1449.pdf) Multigrid (MG) and Local Refinement for Elliptic Partial Differential Equations, by Klaus Stüben

In [ ]:
include("../math405.jl")

### Direct Solvers for 2D, 3D Elliptic Problems

This is a very brief review of [WS03](notes/WS03-FDiff2D.ipynb)

We will restrict attention to the $d$-dimensional Laplace equation, 
$$\begin{aligned}
    -\Delta u = f, \qquad x \in \Omega, \\ 
        u = 0, & x \in \partial \Omega,
\end{aligned}$$
where $\Omega = (0, 1)^d$ and $\partial\Omega$ denotes its boundary. 

Discretisation: 1D grid $x_n = n/N, n = 0, \dots, N$, then
$$
    U_{\bf n} = U_{n_1 \cdots n_d} \approx u(x_{n_1}, \dots, x_{n_d})
$$

$$
    - \Delta_h U = F,
$$
where $F_{\bf n} = f(x_{n_1}, \dots, x_{n_d})$ and 
$$
    \Delta_hU_{{\bf n}} = \sum_{i = 1}^d \frac{U_{{\bf n} + {\bf e}_i} - 2 U_{{\bf n}} + U_{{\bf n} - {\bf e}_i}}{h^2}
    = \Big( D_h^2 \otimes I \otimes \cdots \otimes I
            + I \otimes D_h^2 \otimes I \otimes \cdots \otimes I
            + \cdots
            + I \otimes \cdots \otimes I \otimes D_h^2 \Big) U_{{\bf n}}.
$$

The cost vs error doesn't look good: 


|dimension | error     | system size | cost - order  |
|----------|-----------|-------------|---------------|
|  1       |  N^-2     | N   x N     |  N            |
|  2       |  N^-2     | N^2 x N^2   |  N^3          |
|  3       |  N^-2     | N^3 x N^3   |  N^6          |

Because sparse solvers are very highly optimized, 2D tends to be just fine, while 3D is often no longer tractable for "real-world" problems. 

In [ ]:
# this implementation uses dirichlet boundary conditions only.
L1d(N) = sparse(Tridiagonal(-N^2 * ones(N-2), 2*N^2*ones(N-1), -N^2*ones(N-2)))
Id(N) = sparse(I, (N-1,N-1))
L2d(N) = sparse(kron(L1d(N), Id(N)) + kron(Id(N), L1d(N)))
L3d(N) = sparse(kron(L1d(N), Id(N), Id(N)) + kron(Id(N), L1d(N), Id(N)) + kron(Id(N), Id(N), L1d(N)))

In [ ]:
NN_timings = (2).^(2:9)
timings_direct = zeros(3, length(NN_timings))
for (iN, N) in enumerate(NN_timings), (d, Lfun) in enumerate((L1d, L2d, L3d))
    if N > 2^6 && d == 3; timings_direct[d, iN] = NaN; continue; end 
    timings_direct[d, iN] = (@elapsed (Lfun(N) \ ones((N-1)^d)))
end

In [ ]:
P = plot(; size = (400, 300), yscale = :log10, xscale = :log10, legend = :outertopright, ylims = (1e-5, 1e1)) 
C = [3e-6, 1e-7, 1e-9]; pred = [1, 3, 6]; NN = NN_timings[2:end]
for d = 1:3
    plot!(NN, timings_direct[d, 2:end], c = d, label = "d = $d", lw=3, ms=6, m=:o)
    plot!(NN, C[d] * NN.^pred[d], c=d, ls=:dash, label = "", lw=2)
end
plot!(NN, 1e-6 * NN.^2, c=2, ls=:dot, label = "", lw=1)

### Iterative Solvers : Motivation

For the laplace equation, the finite difference system matrix $A$ is symmetric, positive definite (SPD). In this case, solving $Ax = b$ is equivalent to
$$
    x = \arg\min \Phi(x) := \frac{1}{2} x^T A x - x^T b.
$$
**Proof:** Easy exercise, show that $\nabla \Phi(x) = Ax - b$

**IDEA:** Use steepest descent to solve this!

$\nabla \Phi(x) = Ax - b$, hence 
$$
    x_{n+1} = x_n - \alpha_n (A x_n - b)
$$
Minimize $\Phi(x_{n+1})$ with respect to $\alpha_n$, ..., a bit of algebra, ..., 
$$ 
    x_{n+1} = x_n - \frac{g_n^T g_n}{g_n^T A g_n} g_n,  \qquad g_n := A x_n - b.
$$

**KEY OBSERVATIONS:**
* We only need $A x$ but not $A$ itself!
* No storage required and cost of $A x$ is $O(N^d)$ for all $d$!
* Convergence: can prove (nontrivial!)
$$
    \| x_n - x \| \leq \kappa \Big( \frac{\kappa - 1}{\kappa + 1} \Big)^n \|x_0 - x\|
$$
where $\kappa = {\rm cond}(A)$. 
* For the Laplace equation: $\kappa \approx N^2$.

In [ ]:
"2D discrete laplace operator in matrix format"
function laplace_mat(V)  
    N = size(V, 1)-1
    AxV = zeros(N+1, N+1)
    for n1 = 2:N, n2 = 2:N
        AxV[n1, n2] = N^2 * (- V[n1+1,n2] - V[n1-1,n2] - V[n1,n2+1] - V[n1,n2-1] + 4 * V[n1,n2])
    end
    return AxV 
end

"Apply `nsteps` steepest descent iterations"
function steepest_descent(R, nsteps)
    W = zeros(size(R))
    for n = 1:nsteps
        G = laplace_mat(W) - R 
        W -= norm(G)^2 / dot(G, laplace_mat(G)) * G
    end
    return W 
end

In [ ]:
N = 32
A = L2d(N) 
kappa = cond(Matrix(A)); @show kappa 
u = A \ ones((N-1)^2)
F = zeros(N+1, N+1); F[2:N, 2:N] .= 1
NSTEPS = 20:20:400
Err = [ norm(steepest_descent(F, nsteps)[2:N,2:N][:]-u, Inf) for nsteps in NSTEPS ]
plot(NSTEPS, Err, lw=3, label = "||x_n - x||", yscale = :log10, m=:o, ms=5, size = (400, 300), title = "N = $N")
q = (kappa - 1) / (kappa + 1)
plot!(NSTEPS[10:end], 0.08 * q.^NSTEPS[10:end], c=:black, lw=2, ls=:dash, label = "predicted")

## Krylov Subspace Methods

There are many beautiful and insightful ways to motivate and introduce Krylov subspace methods ... all of them take quite a lot of legwork. Since we don't have that time here, I'll be extremely "sparse". Maybe somebody wants to pickup this topic for a presentation?





**Idea:** starting from gradient descent, 
$$
    x_{n+1} = x_n - \alpha_n \nabla \Phi(x_n) =  x_n - \alpha_n (A x_n - b)
$$

This means that $x_{n+1}$ must be a linear combinatin of $b, A x_0, \dots, A x_n$ and by induction, if we start with $x_0 = 0$, then it must be a linear combination of 
$$
    b, A b, A^2 b, \dots, A^n b
$$
Let's just remember these vectors, and form a *Krylov subspace* 
$$
    K_n := {\rm span}\big\{ b, Ab, A^2 b, \dots, A^n b \big\}.
$$
and then choose the "optimal" approximate solution from this subspace:
$$
    x_n^{\rm cg} := \arg\min_{K_n} \Phi(x)
$$


### The Conjugate Gradient Method

**Non-trivial Observation 1** : $x_n := x_n^{\rm cg}$ satisfies a 3-term recurrance. I.e., we do not need to remember $K_n$. By enlarging the state space from $x_n$ to $(x_n, r_n, d_n)$ we can write it as a convenient 2-term recurrance:  

$$\begin{aligned}
    d_0 &= b - A x_0 \\ 
    x_{n+1} &= x_n + \alpha_n d_n \\ 
    r_{n+1} &= r_n - \alpha_n A d_n \\ 
    d_{n+1} &= r_{n+1} + \beta_{n+1} d_n \\ 
\end{aligned}$$
where 
$$\begin{aligned}
    \alpha_{n+1} &= \frac{r_n^T r_n}{d_n^T A d_n}, \\ 
    \beta_{n+1} &= \frac{r_{n+1}^T r_{n+1}}{r_n^T r_n}.
\end{aligned}$$

**Proof:** non-trivial, requires a lot of work.

**Non-trivial Observation 2:** 
$$
    \| x_n - x \| \leq \kappa \bigg( \frac{\sqrt{\kappa} - 1}{\sqrt{\kappa + 1}}\bigg)^n \| x_0 - x \|.
$$


**Cost Comparison:** If $\epsilon_n = \|x_n - x\|$ (for CG or SD) then 

$$\begin{aligned}
    n &\approx \kappa |\log \epsilon_n|, \qquad \text{SD} \\ 
    n &\approx \sqrt{\kappa} |\log \epsilon_n|, \qquad \text{CG}.
\end{aligned}
$$

For the Laplace operator: $\kappa \approx N^2$, i.e. 
$$\begin{aligned}
    n &\approx N^2 |\log \epsilon_n|, \qquad \text{SD} \\ 
    n &\approx N |\log \epsilon_n|, \qquad \text{CG}.
\end{aligned}
$$

<!-- Cost of one iteration is $O(N^d)$, discretisation error $\epsilon$ is $N^{-2}$ hence
$$\begin{aligned}
    {\rm Cost}  &\approx \epsilon^{-1-d/2} |\log\epsilon|, \qquad \text{SD}, \\ 
    {\rm Cost}  &\approx \epsilon^{-1/2-d/2} |\log\epsilon|, \qquad \text{CG}.
\end{aligned} $$ -->

In [ ]:
using IterativeSolvers, LinearMaps

# Julia's built-in iterative solvers require us to use (abstract) matrices and vectors
# for simplicity we therefore convert between matrix and vector formulation: 

mat2vec(V) = V[2:end-1, 2:end-1][:]

function vec2mat(v)
    N = round(Int, sqrt(length(v))) + 1
    V = zeros(N+1, N+1)
    V[2:N, 2:N] .= reshape(v, N-1, N-1)
    return V
end

laplace_vec = mat2vec ∘ laplace_mat ∘ vec2mat 


In [ ]:
N = 20
L = LinearMap(laplace_vec, (N-1)^2; issymmetric=true, ismutating=false)
F = mat2vec(ones(N+1, N+1))
X = vec2mat( cg(L, F) )
plot(surface(X), contourf(X), size = (600, 200))

In [ ]:
# Convergence test 
N = 128
U = vec2mat(L2d(N)  \ ones((N-1)^2))
L = LinearMap(laplace_vec, (N-1)^2; issymmetric=true, ismutating=false)
F = zeros(N+1, N+1); F[2:N,2:N] .= 1; b = mat2vec(ones(N+1, N+1))
NSTEPS = 20:20:200
ErrSD = [ norm(steepest_descent(F, nsteps)-U, Inf) for nsteps in NSTEPS ]
ErrCG = [ norm(vec2mat( cg(L, b; maxiter = nsteps) ) - U, Inf) for nsteps in NSTEPS ]

plot(NSTEPS, ErrSD, lw=3, label = "SD", yscale = :log10, m=:o, ms=5, size = (400, 300), title = "N = $N")
plot!(NSTEPS, ErrCG, lw=3, label = "CG", m=:o, ms=:5)

But despite the excellent performance of CG, it still deteriorates rapidly with increasing system size! We end up with the same scaling as a direct solver! (but much lower memory requirement!)

In [ ]:
NN = (2).^(3:8) 
num_cg_iters = _N -> cg(LinearMap(laplace_vec, (_N-1)^2; issymmetric=true, ismutating=false), 
                        mat2vec(ones(_N+1, _N+1)), log = true)[2].iters
plot(NN, num_cg_iters.(NN), lw = 3, m = :o, ms=7, label = "# iterations", 
     ylabel = "N", yscale = :log10, xscale = :log10, legend = :topleft, size = (400, 250))
plot!(NN[3:end], NN[3:end], c=:black, label = L"\sim N \propto \sqrt{\kappa}", s=:dash, lw=2)


In [ ]:
# Let's add CG to our timings test!
solve_cg2 = N -> cg(LinearMap(laplace_vec, (N-1)^2; issymmetric=true, ismutating=false), 
                   mat2vec(ones(N+1, N+1)))
solve_cg3 = N -> cg(L3d(N), ones((N-1)^3)) # (don't have a matrix-free version implemented, sorry)
timings_cg2 = [ (@elapsed solve_cg2(N)) for N in NN_timings ]
timings_cg3 = [[ (@elapsed solve_cg3(N)) for N in NN_timings[1:end-2] ]; [NaN,NaN] ]; 

In [ ]:
P = plot(; size = (400, 300), yscale = :log10, xscale = :log10, legend = :outertopright, ylims = (7e-5, 3e1)) 
for d = 2:3
    plot!(NN_timings[2:end], timings_direct[d, 2:end], c = d, label = "direct $(d)D", lw=3, ms=6, m=:o)
end
plot!(NN_timings[2:end], timings_cg2[2:end], c=4, label = "CG 2D", lw=3, ms=6, m=:o)
plot!(NN_timings[2:end], timings_cg3[2:end], c=5, label = "CG 3D", lw=3, ms=6, m=:o)
plot!(NN_timings[2:end], 5e-8 * NN_timings[2:end].^3, c=:black, ls=:dash, label = "", lw=2)
plot!(NN_timings[2:end-2], 5e-8 * NN_timings[2:end-2].^4, c=:black, ls=:dash, label = "", lw=2)


Scaling of direct vs CG: 

|  dimension | direct    |  CG      |
|------------|-----------|----------|
|    1       | N         |  N^2 log ϵ     |
|    2       | N^3 (N^2) |  N^3 log ϵ     |
|    3       | N^6       |  N^4 log ϵ     |




## Linear Scaling!

**Motivation: Linear Scaling Algorithms** 
* Thomas algorithm for 1D FD problems: cost = $O(N)$
* FFT for fourier spectral methods: cost = $O(N^d \log N)$ in all dimensions.

Linear scaling or close to linear scaling is one of the main goals of scientific computing!

## The Cascadic Multigrid Method

![](figs/cascadic_mg.gif)

Cascadic Multi-grid in Pseudocode:
```
h = h0
U[h] = solve(h)
while h > h_fine
    Uc = U[h]          # remember the solution from the previous grid
    h <- h/2           # refine the grid
    Ui = prolong(Uc)   # "prolong" Uc onto the finer grid
    U[h] = cg( init = Ui )    # solve in fine grid with good initial guess
end
```

In [ ]:
"prolong a coarse-grid function to a fine-grid function"
function mg_prolong(V)
    N = size(V, 1) - 1
    Nf = 2 * N
    Vf = zeros(Nf+1, Nf+1)
    for n1 = 2:N, n2 = 2:N
        n1f = 2 * (n1-1) + 1; n2f = 2 *(n2-1) + 1
        Vf[n1f, n2f] = V[n1, n2]
        Vf[n1f+1, n2f] += 0.5 * V[n1, n2]
        Vf[n1f-1, n2f] += 0.5 * V[n1, n2]
        Vf[n1f, n2f+1] += 0.5 * V[n1, n2]
        Vf[n1f, n2f-1] += 0.5 * V[n1, n2]
        Vf[n1f+1, n2f+1] += 0.25 * V[n1, n2]
        Vf[n1f+1, n2f-1] += 0.25 * V[n1, n2]
        Vf[n1f-1, n2f+1] += 0.25 * V[n1, n2]
        Vf[n1f-1, n2f-1] += 0.25 * V[n1, n2]
    end
    return Vf 
end

In [ ]:
cg_iters(U0, nsteps) = (N = size(U0, 1)-1; 
                        cg!(mat2vec(U0), LinearMap(laplace_vec, (N-1)^2; issymmetric=true, ismutating=false), 
                            mat2vec(ones(N+1, N+1)), maxiter = nsteps) |> vec2mat)
function cascadic_mg(levels, niter; N = 2)
    U = zeros(N÷2+1,N÷2+1)
    for l = 1:levels
        U = mg_prolong(U)
        U = cg_iters(U, niter)
    end
    return U
end

In [ ]:
U = cascadic_mg(4, 10)
plot(surface(U), contourf(U), size = (600, 200))

The number of levels determines $h$ i.e. the accuracy of the 
finite difference scheme. The only "tuning" parameter we have 
available to tweak the accuracy of the linear solver is 
`niter` the number of CG iterations at each level.

In [ ]:
# Convergence Test 
NITER = [5, 10, 20, 40, 1_000]
LEVELS = collect(2:6)
ERRs = zeros(length(LEVELS), length(NITER))
for (il, levels) in enumerate(LEVELS)
    U = cascadic_mg(levels, 2)
    N = size(U, 1) - 1
    Ux = vec2mat(L2d(N) \ ones((N-1)^2))  # direct FD solution 
    for (iit, niter) in enumerate(NITER)
        Umg = cascadic_mg(levels, niter)  # cascaded multigrid solution
        ERRs[il, iit] = norm(Ux - Umg, Inf)
    end    
end


In [ ]:
header = [ ["#lev"]; ["#iter = $(NITER[1])"]; string.(NITER[2:end-1]); ["Inf"] ]
data = [LEVELS ERRs]
pretty_table(data, header; 
             formatters = (v,i,j) -> (j == 1 ? Int(v) : (@sprintf("%.2e", v))) )

### Cascadic Multigrid - Cost Analysis

* Cost per iteration: $O(N_l^d)$ x #iterations; hence overall cost is 
$$
   {\rm COST} \approx \text{\# iter} \times \sum_l N_l^d \approx \text{\# iter} \times N_{\rm fine}^d.
$$
* Our convergence analysis suggests (and one can prove this!) the number of required iterations scales as $\log \epsilon$ (desired accuracy)
* We want $\epsilon = N^{-2}$ which suggests choosing $\# iter = C \log N^2$. 
* Overall cost:
$$
    {\rm COST}(N) \approx N^2 \log N
$$

**Cascadic MG is almost a linear scaling method!**

Our implementation is a bit naive and needs to be tweaked in a variety of ways, but the basic idea is a good one.

In [ ]:
# Confirming that claim ..
NN = (2).^(2:12)
solve_cmg = N -> cascadic_mg(round(Int, log2(N)), round(Int, 5 * log10(N)))
tim_direct = [ (L = L2d(N); b = ones((N-1)^2); @elapsed (L \ b)) for N in NN ]
tim_cg2 = [[ (@elapsed solve_cg2(N)) for N in NN[1:end-2] ]; [NaN, NaN]]
tim_cmg = [ (@elapsed solve_cmg(N)) for N in NN ];

In [ ]:
plot(; size = (400, 300), yscale = :log10, xscale = :log10, legend = :outertopright, ylims = (7e-5, 3e2)) 
plot!(NN[2:end], tim_direct[2:end], c = 1, label = "direct 2D", lw=3, ms=6, m=:o)
plot!(NN[2:end], tim_cg2[2:end], c=2, label = "CG 2D", lw=3, ms=6, m=:o)
plot!(NN[2:end], tim_cmg[2:end], c=3, label = "C-MG 2D", lw=3, ms=6, m=:o)
plot!(NN[2:end], 1e-8 * NN[2:end].^3, c=:black, ls=:dash, label = "", lw=2)
plot!(NN[2:end], 1e-6 * NN[2:end].^2, c=:black, ls=:dash, label = "", lw=2)

### A deeper (slightly less superficial) look at why C-MG works well.

**Basic message:** 
* When prolonging $U[h] \to U[h/2]$, the remaining error is a high-frequency error. 
* High frequency errors are the first that get dampened by typical iterative solvers.

In [ ]:
plots = []
plargs = (lw=0.2, colorbar=false, xticks = [], yticks = [])
for N in [4, 8, 16]
    Uc = vec2mat( L2d(N) \ ones((N-1)^2) )
    Ui = mg_prolong(Uc)
    Uf = vec2mat( L2d(2*N) \ ones((2*N-1)^2) )
    append!(plots, [contourf(Uc; plargs...), contourf(Uf; plargs...), contourf(Uf - Ui; plargs..., lw=0)])
end

In [ ]:
plot(plots..., size = (600, 600))

## Conclusions / Take-home Messages

* Be aware of the scaling of computational cost and memory requirements for 2D and 3D problems!
* Iterative solvers are a useful choice for 2D problems and a necessary choice for 3D problems!
* Linear scaling is possible but often difficult to obtain and may require deep insight into the problem being solved.
* Not all problems can be solved by multi-grid methods! Different problems require different ideas to achieve linear scaling complexity. 
* Multigrid is often more naturally interpreted as a preconditioner. See separate recorded lecture or workshop (TBD!) **Highly recommended for those of you interested in scientific computing!**